In [1]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardMarkup, KeyboardButton
from telepot.delegate import (per_chat_id, per_application, call, create_open, pave_event_space)
from pprint import pprint
import sys
import time
import pandas as pd
#%run Result_Word2Vec.ipynb
%run chatbot.ipynb
#%run w2v_ontology.ipynb

<h3>Class of Bot</h3>

In [2]:
class ChatBox(telepot.DelegatorBot):
    def __init__(self, token, owner_id):
        self._owner_id = owner_id
        self._seen = set()
        self._store = MessageStore()
        
        super(ChatBox, self).__init__(token, [
            pave_event_space()(
                per_chat_id(), create_open, MessageHandler, self._store, timeout=20),
        ])

<h1>Start Bot Telegram</h1>

In [ ]:
OWNER_ID = 102199992
TOKEN = '2067569208:AAHYKIRPZJ_eKtEFAtAmcFOWGlOMyRbGsXI'

bot = ChatBox(TOKEN, OWNER_ID)
MessageLoop(bot).run_as_thread()
print('Listening ...')

while 1:
    time.sleep(3)

Listening ...


ERROR:root:on_close() called due to IdleTerminate: 20


RESULTS:  [(46997036, 0, ['numpy', 'matrix', 'multiplication'], 'numpy matrix multiplication row-wise'), (62927349, 0, ['matrix', 'tkinter'], 'Building matrix GUI with tkinter'), (41208590, 0, ['numpy', 'theano'], 'How to use integer values of a matrix as index for another matrix using numpy or Theano?'), (61521226, 0, ['numpy', 'matrix'], 'numpy.dot function matrix multiplication: Numpy multiplies AxB matrix and 1xB matrix as AxB Matrix and Bx1 Matrix'), (64955936, 0, ['numpy', 'matrix', 'error-handling', 'matrix-inverse'], 'Python: Why do I get error message when I try to calculate the inverse of my 2x2-matrix (Hessian)?'), (46117275, 0, ['list', 'matrix', '2d'], '2d list / matrix in python'), (60271755, 0, ['numpy'], 'Numpy matrix flattens to a matrix'), (64510699, 0, ['numpy', 'matrix', 'matrix-multiplication', 'tensordot'], 'Multiplication of 2D array and 3D array'), (38680508, 0, ['python-3.x', 'numpy', 'scipy'], 'How to vstack efficiently a sequence of large numpy array chunks?'

ERROR:root:on_close() called due to IdleTerminate: 20
ERROR:root:on_close() called due to IdleTerminate: 20
ERROR:root:on_close() called due to IdleTerminate: 20


RESULTS:  [(60473294, 0.1098869802219295, ['pandas', 'dummy-variable', 'dummy-data'], 'Create dummy coded columns for a column and concatenate it to the dataset'), (44357970, 0.10690648522595564, ['numpy', 'tensorflow', 'linear-algebra'], 'How to implement tf.space_to_depth with numpy?'), (49517830, 0.10009791765923194, ['regex', 'pandas'], 'Incorrect regex identification using pandas'), (41962066, 0.07921236140940184, ['scipy', 'spatial', 'delaunay', 'simplex'], 'How to select from the output of scipy Delaunay triangulation only simplices under certain volume (or under total line length)?'), (6207557, 0.06951895946835213, ['django', 'django-models', 'django-queryset'], 'How to reduce queries in django model has_relation method?'), (44148776, 0.06615154392567552, ['web-scraping', 'beautifulsoup', 'scrapy', 'urllib'], 'Scraping searchable online dictionary'), (58060307, 0.06350198575012807, ['pandas'], 'Filter columns by same value'), (51062770, 0.05640116126000585, ['pandas'], 'Python 

ERROR:root:on_close() called due to IdleTerminate: 20


STAMPA RESULTS:  [(27029020, 0, ['nlp', 'nltk'], 'Too many values to unpack using NLTK and Pandas in Python'), (60847480, 0, ['nltk'], 'what should i do to remove the error of importing nltk as i have already installed it?'), (64945076, 0, ['heroku'], "Using Natural Language Tool Kit with Django on Heroku - - Error: 'nltk.txt' not found"), (24107444, 0, ['flask', 'nltk'], "LookupError: Resource 'corpora/stopwords' not found")]


ERROR:root:on_close() called due to IdleTerminate: 20
ERROR:root:on_close() called due to IdleTerminate: 20
